eksik sonuc verdi.

yapılan işlemler onucu skor yükselmedi. Lokasyon, ve keyword gibi değişkenler concatlanıp bakılacak.
Skoru yükselten asıl kaynağın model kurulması ve çalıştırılması olduğu anlaşıldı....

In [1]:
import numpy as np
import pandas as pd
import operator
from wordcloud import STOPWORDS
import string

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv') 

In [4]:
import re
def clean(tweet): 
    
    # Special characters
   
   
    # Urls
    tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)
        
    # Words with punctuations and special characters
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`"
    for p in punctuations:
        tweet = tweet.replace(p, f' {p} ')
        
    # ... and ..
    tweet = tweet.replace('...', ' ... ')
    if '...' not in tweet:
        tweet = tweet.replace('..', ' ... ')      
        
    # Acronyms
    tweet = re.sub(r"MH370", "Malaysia Airlines Flight 370", tweet)
    tweet = re.sub(r"mÌ¼sica", "music", tweet)
    tweet = re.sub(r"okwx", "Oklahoma City Weather", tweet)
    tweet = re.sub(r"arwx", "Arkansas Weather", tweet)    
    tweet = re.sub(r"gawx", "Georgia Weather", tweet)  
    tweet = re.sub(r"scwx", "South Carolina Weather", tweet)  
    tweet = re.sub(r"cawx", "California Weather", tweet)
    tweet = re.sub(r"tnwx", "Tennessee Weather", tweet)
    tweet = re.sub(r"azwx", "Arizona Weather", tweet)  
    tweet = re.sub(r"alwx", "Alabama Weather", tweet)
    tweet = re.sub(r"wordpressdotcom", "wordpress", tweet)    
    tweet = re.sub(r"usNWSgov", "United States National Weather Service", tweet)
    tweet = re.sub(r"Suruc", "Sanliurfa", tweet)   
    
    # Grouping same words without embeddings
    tweet = re.sub(r"Bestnaijamade", "bestnaijamade", tweet)
    tweet = re.sub(r"SOUDELOR", "Soudelor", tweet)
    
    return tweet


df_train['text_cleaned'] = df_train['text'].apply(lambda s : clean(s))
df_test['text_cleaned'] = df_test['text'].apply(lambda s : clean(s))





In [7]:
df_mislabeled = df_train.groupby(['text']).nunique().sort_values(by='target', ascending=False)
df_mislabeled = df_mislabeled[df_mislabeled['target'] > 1]['target']
df_mislabeled.index.tolist()

['like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit',
 'Hellfire! We don\x89Ûªt even want to think about it or mention it so let\x89Ûªs not do anything that leads to it #islam!',
 "The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'",
 'In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!',
 'To fight bioterrorism sir.',
 'Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her\n \n#FARRAKHAN #QUOTE',
 '#foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption',
 '#Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect',
 'He came to a land which was engulfed in tribal war and turned it into a land 

In [8]:
df_train['target_relabeled'] = df_train['target'].copy() 

df_train.loc[df_train['text'] == 'like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == 'Hellfire is surrounded by desires so be careful and donÛªt let your desires control you! #Afterlife', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == 'To fight bioterrorism sir.', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == '.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4', 'target_relabeled'] = 1
df_train.loc[df_train['text'] == 'CLEARED:incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring', 'target_relabeled'] = 1
df_train.loc[df_train['text'] == '#foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == 'In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == 'Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her\n \n#FARRAKHAN #QUOTE', 'target_relabeled'] = 1
df_train.loc[df_train['text'] == 'RT NotExplained: The only known image of infamous hijacker D.B. Cooper. http://t.co/JlzK2HdeTG', 'target_relabeled'] = 1
df_train.loc[df_train['text'] == "Mmmmmm I'm burning.... I'm burning buildings I'm building.... Oooooohhhh oooh ooh...", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "wowo--=== 12000 Nigerian refugees repatriated from Cameroon", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "Hellfire! We donÛªt even want to think about it or mention it so letÛªs not do anything that leads to it #islam!", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "Caution: breathing may be hazardous to your health.", 'target_relabeled'] = 1
df_train.loc[df_train['text'] == "I Pledge Allegiance To The P.O.P.E. And The Burning Buildings of Epic City. ??????", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "#Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "that horrible sinking feeling when youÛªve been at home on your phone for a while and you realise its been on 3G this whole time", 'target_relabeled'] = 0

In [9]:
# word_count
df_train['word_count'] = df_train['text'].apply(lambda x: len(str(x).split()))
df_test['word_count'] = df_test['text'].apply(lambda x: len(str(x).split()))

# unique_word_count
df_train['unique_word_count'] = df_train['text'].apply(lambda x: len(set(str(x).split())))
df_test['unique_word_count'] = df_test['text'].apply(lambda x: len(set(str(x).split())))

# stop_word_count
df_train['stop_word_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
df_test['stop_word_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))

# url_count
df_train['url_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
df_test['url_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
df_train['mean_word_length'] = df_train['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test['mean_word_length'] = df_test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# char_count
df_train['char_count'] = df_train['text'].apply(lambda x: len(str(x)))
df_test['char_count'] = df_test['text'].apply(lambda x: len(str(x)))

# punctuation_count
df_train['punctuation_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test['punctuation_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
df_train['hashtag_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
df_test['hashtag_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
df_train['mention_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
df_test['mention_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split( df_train[['text_cleaned', "word_count", "unique_word_count","stop_word_count"]]
                                                     , df_train["target_relabeled"], test_size=0.2,
                                                       random_state = 1)

In [11]:
train_x1 = train_x["text_cleaned"]
train_x2 = train_x["word_count"]

test_x1 = test_x["text_cleaned"]
test_x2 = test_x["word_count"]

In [12]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit_transform(train_x1)


<6090x14841 sparse matrix of type '<class 'numpy.float64'>'
	with 79291 stored elements in Compressed Sparse Row format>

In [13]:
x_train_tf_idf_word1 = tf_idf_word_vectorizer.transform(train_x1)

In [14]:
from scipy.sparse import coo_matrix, hstack

In [45]:
x_train_tf_idf_word2 = coo_matrix(train_x2)

In [50]:
x_train_tf_idf_word2 = x_train_tf_idf_word2.reshape(6090,1)

In [131]:
x_train_tf_idf_word = hstack([x_train_tf_idf_word1,x_train_tf_idf_word2])

In [133]:
x_train_tf_idf_word = x_train_tf_idf_word.toarray()

In [137]:
type(x_train_tf_idf_word)

numpy.ndarray

In [63]:
x_test_tf_idf_word1 = tf_idf_word_vectorizer.transform(test_x1)

In [64]:
x_test_tf_idf_word2 = coo_matrix(test_x2)

In [65]:
x_test_tf_idf_word2 = x_test_tf_idf_word2.reshape(1523,1)

In [118]:
x_test_tf_idf_word = hstack([x_test_tf_idf_word1, x_test_tf_idf_word2])

In [135]:
x_test_tf_idf_word = x_test_tf_idf_word.toarray()

In [ ]:
type(x_test_tf_idf_word)

In [144]:
test1 = df_test["text_cleaned"]
test1 = tf_idf_word_vectorizer.transform(test1)

In [147]:
np.shape(test1)

(3263, 14841)

In [148]:
test2 = df_test["word_count"]

In [149]:
test2 = coo_matrix(test2)

In [150]:
test2 = test2.reshape(3263,1)

In [151]:
test = hstack([test1, test2])


In [152]:
test = test.toarray()

In [153]:
type(test)

numpy.ndarray

In [187]:
from keras.utils import to_categorical
from keras import models
from keras import layers
import numpy as np


from keras.models import Sequential

# Input - Layer
model = Sequential()
model.add(layers.Dense(10, activation = "relu", input_shape=(14842, )))
# Hidden - Layers
model.add(layers.Dense(4, activation = "relu"))

model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_51 (Dense)             (None, 10)                148430    
_________________________________________________________________
dense_52 (Dense)             (None, 4)                 44        
_________________________________________________________________
dense_53 (Dense)             (None, 1)                 5         
Total params: 148,479
Trainable params: 148,479
Non-trainable params: 0
_________________________________________________________________


In [188]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [189]:
results = model.fit(
 x_train_tf_idf_word, train_y,
 epochs= 12,
 batch_size = 150,
 validation_data = (x_test_tf_idf_word, test_y)
)

Train on 6090 samples, validate on 1523 samples
Epoch 1/12
6090/6090 [==============================] - 2s 255us/step - loss: 0.6892 - accuracy: 0.5575 - val_loss: 0.6822 - val_accuracy: 0.5817
Epoch 2/12
6090/6090 [==============================] - 1s 199us/step - loss: 0.6782 - accuracy: 0.5695 - val_loss: 0.6711 - val_accuracy: 0.5817
Epoch 3/12
6090/6090 [==============================] - 1s 199us/step - loss: 0.6621 - accuracy: 0.5849 - val_loss: 0.6543 - val_accuracy: 0.6205
Epoch 4/12
6090/6090 [==============================] - 1s 202us/step - loss: 0.6370 - accuracy: 0.6695 - val_loss: 0.6305 - val_accuracy: 0.7098
Epoch 5/12
6090/6090 [==============================] - 1s 200us/step - loss: 0.6013 - accuracy: 0.7874 - val_loss: 0.6013 - val_accuracy: 0.7912
Epoch 6/12
6090/6090 [==============================] - 1s 202us/step - loss: 0.5566 - accuracy: 0.8383 - val_loss: 0.5663 - val_accuracy: 0.8004
Epoch 7/12
6090/6090 [==============================] - 1s 202us/step - loss

In [190]:
y_pred = model.predict(test)
y_pred = (y_pred > 0.5)
y_pred1 = pd.DataFrame(data = y_pred, index = range(3263), columns=['target'])
y_pred1["target"] = y_pred1["target"].astype(int)

In [191]:
submission = pd.read_csv('sample_submission.csv') 
submission= submission[["id"]]
final_submission=pd.concat([submission,y_pred1],axis=1)
final_submission.to_csv('11062020_12.csv',index=False)


In [156]:
final_submission.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1


In [51]:
np.shape(final_submission)

(3263, 2)

In [270]:
np.shape(x_train_tf_idf_word)

(5709, 7123)